<a href="https://colab.research.google.com/github/PapaKwansa/Lab_report_overleaf/blob/master/Analytical_Solution_LG_edited.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def C_matrix(x,y,z,a,b,c):
  return np.array([
    [x - a, y - b, z - c],  # Vertex 1
    [x + a, y - b, z - c],  # Vertex 2
    [x + a, y + b, z - c],  # Vertex 3
    [x - a, y + b, z - c],  # Vertex 4
    [x - a, y + b, z + c],  # Vertex 5
    [x - a, y - b, z + c],  # Vertex 6
    [x + a, y - b, z + c],  # Vertex 7
    [x + a, y + b, z + c]   # Vertex 8
])

def R_n(n,C):
  return np.linalg.norm(C[n])


def delta(i,j):
  return int(i==j)

deltamod = np.array([[0,1,0],[0,0,1],[1,0,0]])

def V0(x,y,z,a,b,c):
  C = C_matrix(x,y,z,a,b,c)
  R_n_list = [R_n(n,C) for n in range(8)]
  result = 0
  for n in range(8):
    for i in range(3):
      result += C[n,i]*C[n,(i+1)%3]*np.log(R_n_list[n]+C[n,(i+2)%3]) - (C[n,i]**2)/2*np.arctan((C[n,(i+1)%3]*C[n,(i+2)%3])/(C[n,i]*R_n_list[n]))
    result *= ((-1)**(n+1))
  return result

def v10(x,y,z,a,b,c):
   C = C_matrix(x,y,z,a,b,c)
   R_n_list = [R_n(n,C) for n in range(8)]
   result = np.zeros(3)
   for i in range(3):
    for n in range(8):
        result[i] += ((-1)**(n+1)) * ((C[n,(i+1)%3])*np.log(R_n_list[n]+C[n,(i+2)%3])+C[n,(i+2)%3]*np.log(R_n_list[n]+C[n,(i+1)%3]) - (C[n,i])*np.arctan((C[n,(i+1)%3]*C[n,(i+2)%3])/(C[n,i]*R_n_list[n])))
   return result

def v20(x,y,z,a,b,c):
   C = C_matrix(x,y,z,a,b,c)
   R_n_list = [R_n(n,C) for n in range(8)]
   result = np.zeros([3,3])
   for i in range(3):
    for j in range(3):
      for n in range(8):
        result[i,j] += ((-1)**(n+1)) * ((deltamod[i,j] * np.log(C[n, (j+1)%3] + R_n_list[n])) + deltamod[j,i] * np.log(C[n, (i+1)%3] + R_n_list[n]) - delta(i,j) * np.arctan((C[n,(i+1)%3]*C[n,(i+2)%3])/(C[n,i]*R_n_list[n])))
   return result

def v30(x,y,z,a,b,c):
   C = C_matrix(x,y,z,a,b,c)
   R_n_list = [R_n(n,C) for n in range(8)]
   result = np.zeros([3,3])
   for i in range(3):
    for j in range(3):
      for n in range(8):
        result[i,j] += ((-1)**(n+1)) * (((deltamod[i,j] * (C[n, 2] + delta(i,0) * R_n_list[n]))/ (R_n_list[n] * (C[n, (i+2)%3] + R_n_list[n]))) + ((deltamod[j,i] * (C[n, 2] + delta(j,0) * R_n_list[n]))/ ((R_n_list[n] * (C[n, (j+2)%3] + R_n_list[n])))) - delta(i,j) * ((C[n, 0] * C[n, 1] *((1 - 2 * delta(i, 2)) * ((R_n_list[n]) ** 2) - (C[n, 2]**2))) / (R_n_list[n] * ((C[n, i] **2) * (R_n_list[n] **2)  + (C[n, (i+ 1)% 3] **2) * ((C[n, (i+ 2)% 3]) **2)))))
   return result


# Define the parameters
p = 1e6  # Pressure in the inclusion
nu = 0.25 # The Poisson's ratio
E = 10e9  # Young's modulus
alpha =  0.8 # Biot coefficient
# ec = 5.3051647697e-6 # Characteristic strain, this is the ec for the trial answer.
h =  2500 # Depth of the inclusion
x = -900 # x-coordinate of the observation point
y = 750 # y-coordinate of the observation point
z = 60 # z-coordinate of the observation point
a = 175
b = 25
c = 125
ec = 5.3051647697 * (10 ** -6)

def disp(x,y,z,a,b,c, ec, h):
  result = np.zeros(3)
  V101 = v10(x,y,z-h,a,b,c)
  V102 = v10(x,y,-z-h,a,b,c)
  v201 = v20(x,y,-z-h,a,b,c)
  for i in range(3):
    result[i] = -ec * (V101[i]+ (3 - 4 * nu) * V102[i] - 2 *(1 - 2 * delta(i,2))*z * v201[i,2])
  return result

def disp_prime(x,y,z,a,b,c, ec, h):
  v201 = v20(x,y,z-h,a,b,c)
  v202 = v20(x,y,-z-h,a,b,c)
  v301 = v30(x,y,-z-h,a,b,c)
  result = np.zeros([3,3])
  for i in range(3):
    for j in range(3):
       result[i,j] = -ec * (v201[i,j] + ((3 - 4 * nu) * (1 - 2 * delta(j,2)) - 2 * (1 - 2 * delta(i, 2)) * delta(j,2)) * v202[i,j] + (-2)*(1- 2 * delta(i,2))*(1- 2 * delta(j,2)) * z * v301[i,j])
  return result

def strain(x,y,z,a,b,c, ec, h):
  local_disp_prime = disp_prime(x,y,z,a,b,c, ec, h)
  result = np.zeros([3,3])
  for i in range(3):
    for j in range(3):
      result[i,j] = 0.5 * (local_disp_prime[i,j] + local_disp_prime[j,i])
  return result

#

# Call all the functions for the computation....
def main(x,y,z,a,b,c, ec, nu, h):
  print(disp(x,y,z,a,b,c, ec, h))
  print(disp_prime(x,y,z,a,b,c, ec, h))
  print(strain(x,y,z,a,b,c, ec, h))

main(x,y,z,a,b,c, ec, nu, h)

[-2.81186633e-06  2.35217980e-06 -8.36762309e-06]
[[ 2.16592155e-09  8.03769994e-10  2.43586234e-09]
 [ 8.03769994e-10  2.46213840e-09 -2.04195224e-09]
 [-2.99105826e-09  2.50921802e-09 -1.54672670e-09]]
[[ 2.16592155e-09  8.03769994e-10 -2.77597963e-10]
 [ 8.03769994e-10  2.46213840e-09  2.33632889e-10]
 [-2.77597963e-10  2.33632889e-10 -1.54672670e-09]]
